In [8]:
# Importing library
import requests
import json
import re
import random
import spacy
import string
import scipy
from string import punctuation
from nltk import tokenize
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
import yake
from keybert import KeyBERT
from summarizer import Summarizer
import nltk
import io
from IPython.display import Markdown, display
from random import shuffle
from io import StringIO
nltk.download('stopwords')
nltk.download('popular')
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
import torch
import benepar
nltk.download('punkt')
benepar.download('benepar_en3')
import benepar
parser = benepar.Parser("benepar_en3")
import streamlit as st
import os

# Importing custom python module
import TrueAndFalse
import Generatingblanks
import GeneratingDistractor

[nltk_data] Downloading package stopwords to /Users/jenil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/jenil/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/jenil/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/jenil/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/jenil/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/jenil/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading pac

In [16]:
full_text = open('5g.txt', 'r').read()
mcq=1
fib=0
bool=0
if mcq :
    # Summarizing text
    summarized_text = Generatingblanks.bert_summ(full_text)
    # Keyword selection (Summarized Text)
    kw_summ = Generatingblanks.keyword_selection(summarized_text)
    # Keyword selection (full text)
    kw_full = Generatingblanks.keyword_selection(full_text)
    print(kw_summ)
    # Lowering text from keywords by summarized text
    filtered_keys1=[]
    for keyword in kw_summ :
        if keyword.lower() in summarized_text.lower():
            filtered_keys1.append(keyword)
        
    # Lowering text from keywords by full text      
    filtered_keys2 = []
    for keyword in kw_full  :
        if keyword.lower() in full_text.lower():
            filtered_keys2.append(keyword)

    # Mapping keyword with the sentence (summarized text)
    sentences1 = Generatingblanks.SentenceMapping.tokenize_sentences(summarized_text)
    keyword_sentence_mapping1 = Generatingblanks.SentenceMapping.get_sentences_for_keyword(filtered_keys1, sentences1)

    # Mapping keyword with the sentence (full text)
    sentences2 = Generatingblanks.SentenceMapping.tokenize_sentences(full_text)
    keyword_sentence_mapping2 = Generatingblanks.SentenceMapping.get_sentences_for_keyword(filtered_keys2, sentences2)

    # Combining both the keyword Dictionary
    kws_m =  keyword_sentence_mapping2.copy()
    kws_m.update(keyword_sentence_mapping1)
    print(kws_m)
    GeneratingDistractor.generate_options(kws_m)
    print("done")

# Fill in the blanks
if fib :
    # Summarizing text
    summarized_text = Generatingblanks.bert_summ(full_text)
    # Keyword selection (Summarized Text)
    kw_summ = Generatingblanks.keyword_selection(summarized_text)
    # Keyword selection (full text)
    kw_full = Generatingblanks.keyword_selection(full_text)

    # Lowering text from keywords by summarized text
    filtered_keys1=[]
    for keyword in kw_summ :
        if keyword.lower() in summarized_text.lower():
            filtered_keys1.append(keyword)

    # Lowering text from keywords by full text      
    filtered_keys2 = []
    for keyword in kw_full  :
        if keyword.lower() in full_text.lower():
            filtered_keys2.append(keyword)

    # Mapping keyword with the sentence (summarized text)
    sentences1 = Generatingblanks.SentenceMapping.tokenize_sentences(summarized_text)
    keyword_sentence_mapping1 = Generatingblanks.SentenceMapping.get_sentences_for_keyword(filtered_keys1, sentences1)

    # Mapping keyword with the sentence (full text)
    sentences2 = Generatingblanks.SentenceMapping.tokenize_sentences(full_text)
    keyword_sentence_mapping2 = Generatingblanks.SentenceMapping.get_sentences_for_keyword(filtered_keys2, sentences2)

    # Combining both the keyword Dictionary
    kws_m =  keyword_sentence_mapping2.copy()
    kws_m.update(keyword_sentence_mapping1)

    # Code for generating fill in the blanks as output
    index = 1
    for x,y in kws_m.items(): 
        sentence = y[0]
        pattern = re.compile(x, re.IGNORECASE)
        output = pattern.sub( " _______ ", sentence)
        print("%s)"%(index),output)
        index = index + 1 
        print('Answer is',x)

# True & False selected
if bool :
    # Summarizing the text
    cand_sents = TrueAndFalse.BoolSumText.get_candidate_sents(full_text)
    # Removing quoted sentences
    filter_quotes_and_questions = TrueAndFalse.BoolSumText.preprocess(cand_sents)
    sent_completion_dict = TrueAndFalse.get_sentence_completions(filter_quotes_and_questions)
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    print("hello")
    model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)
    model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')
    print('***********')
    index = 1
    choice_list = ["a)","b)","c)","d)","e)","f)"]
    for key_sentence in sent_completion_dict:
        partial_sentences = sent_completion_dict[key_sentence]
        false_sentences =[]
        print_string = "**%s) True Sentence (from the story) :**"%(str(index))
        print(print_string)
        print("  ",key_sentence)
        for partial_sent in partial_sentences:
            # Generating false statements
            false_sents = TrueAndFalse.GenerateFalseText.generate_sentences(partial_sent,key_sentence)
            false_sentences.extend(false_sents)
        print("  **False Sentences (GPT-2 Generated)**")
        # Printing output for True and False sentences
        for ind,false_sent in enumerate(false_sentences):
            print_string_choices = "**%s** %s"%(choice_list[ind],false_sent)
            #printmd(print_string_choices)
            print_string_choices
        index = index+1
        print('Executing...')
        print("\n\n")


['Coordinate covalent bonding', 'shared electron pairs', 'dative bonding', 'covalent bonding', 'concept of shared', 'electron pair', 'electron', 'coordinate bonds', 'Lewis acid-base theory', 'Lewis', 'bonding', 'shared electron', 'Lewis acid-base', 'Lewis acid', 'Coordinate', 'electron density', 'bonds', 'atom', 'familiar covalent bond', 'Coordinate covalent', 'bonding', 'covalent', 'molecules', 'complexes', 'electron']
done
